In [50]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_log_error
import matplotlib.pyplot as plt
import math


In [71]:
inputs = pd.read_csv('traincleaned.csv')
testinputs = pd.read_csv('testcleaned.csv')
featureimportance = pd.read_csv('feature importance.csv')

inputs = inputs.drop(columns=['SalePrice'])
alldata = pd.concat([inputs, testinputs])
alldata.set_index('Id', inplace=True)
alldata = alldata.fillna(0)
alldata = pd.get_dummies(alldata)
alldata = alldata.drop(columns=['Unnamed: 0'])

inputs = alldata.loc[0:1460,]
inputs = inputs.values

testinputs = alldata.loc[1461:]
testinputs = testinputs.values



In [73]:
featureimportance = featureimportance.rename(columns={'0': 'importance', '1':'feature'})
featureimportance = featureimportance.drop(columns=['Unnamed: 0'])
featureimportance = featureimportance.sort_values(by='importance', ascending=False)

KeyError: "['Unnamed: 0'] not found in axis"

In [53]:
import numpy

prices = pd.read_csv('train.csv')
prices = prices['SalePrice']
prices = np.array(prices)
prices = numpy.log10(prices)
prices

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [54]:
print(inputs.shape, prices.shape)

(1460, 311) (1460,)


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


X_train, X_test, y_train, y_test = train_test_split(inputs, prices, random_state=1)

scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

X_train = X_train_scale
X_test = X_test_scale

inputs = scaler.fit_transform(inputs)


In [44]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[.005, .01, .05, .1, 1, 10], 
                                          'gamma':[.1, .2, .3, .4, .5, .6, .7, .8, .9, 1],
                                          'epsilon':[.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]}
clf = SVR()
clf = GridSearchCV(clf, parameters, cv=5)
clf.fit(X_train, y_train) 

def my_custom_loss_func(X_train_scaled, Y_train_scaled):
    error, M = 0, 0
    for i in range(0, len(Y_train_scaled)):
        z = (Y_train_scaled[i] - M)
        if X_train_scaled[i] > M and Y_train_scaled[i] > M and (X_train_scaled[i] - Y_train_scaled[i]) > 0:
            error_i = (abs(Y_train_scaled[i] - X_train_scaled[i]))**(2*np.exp(z))
        if X_train_scaled[i] > M and Y_train_scaled[i] > M and (X_train_scaled[i] - Y_train_scaled[i]) < 0:
            error_i = -(abs((Y_train_scaled[i] - X_train_scaled[i]))**(2*np.exp(z)))
        if X_train_scaled[i] > M and Y_train_scaled[i] < M:
            error_i = -(abs(Y_train_scaled[i] - X_train_scaled[i]))**(2*np.exp(-z))
    error += error_i
    return error

GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [0.005, 0.01, 0.05, 0.1, 1, 10], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
for C in [.005, .01, .05, .1, 1, 10]:
    
    for gamma in range(1, 10):
        gamma = gamma/10
        
        for epsilon in range(1,10):
            epsilon = epsilon/10
            
            clf = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
            clf.fit(X_train, y_train) 
            testpredictions = clf.predict(X_test)
            trainpredictions = clf.predict(X_train)
            
            testpredictions = 10**testpredictions
            trainpredictions = 10**trainpredictions
            y_test = 10**y_test
            y_train = 10**y_train

            testRMSLE = math.sqrt(mean_squared_log_error(y_test, testpredictions))
            trainRMSLE = math.sqrt(mean_squared_log_error(y_train, trainpredictions))
                
            if RMSLE < .14:
                print(C, gamma, epsilon, testRMSLE, trainRMSLE)
            


0.005 0.1 0.1 0.3406263264199532 0.3219688005839796


C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in power
  
C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in power
  from ipykernel import kernelapp as app
C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in power
  app.launch_new_instance()
C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in power


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [8]:
for C in range(200,300,20):
    
    for degree in range(4, 10):
        
        for epsilon in range(1,10, 2):
            epsilon = epsilon/10
            
            for coef0 in range(2,4):
            
                clf = SVR(kernel='poly', C=C, gamma='auto', degree=degree, epsilon=epsilon, coef0=coef0)
                clf.fit(X_train, y_train) 
                predictions = clf.predict(X_test)

                RMSLE = math.sqrt(mean_squared_log_error(y_test, predictions))
                
                if RMSLE < .14:
                    print(C, degree, epsilon, coef0, RMSLE)
                
                
#C=130,degree=7,epsilon=0.1,coef0=3 0.1359030945302552 first submission
#C=190,degree=9,epsilon=0.9,coef0=2 0.1359030945302552

In [56]:
clf = SVR(kernel='poly', gamma='auto', C=130,degree=7,epsilon=0.1,coef0=3)
clf.fit(inputs, prices) 
testinputs = scaler.fit_transform(testinputs)
predictions = clf.predict(testinputs)



In [24]:
clf = SVR(kernel='rbf', C=50, gamma=0.1, epsilon=0.5)
clf.fit(X_train, y_train) 
predictions = clf.predict(testinputs)

1459

In [57]:
submittest = pd.read_csv('testcleaned.csv')
submittest = submittest[['Id']]
predictions = 10**predictions
submittest['SalePrice'] = predictions
submittest.to_csv('BrandenSubmission.csv', index=False)
submittest

,Id,SalePrice
0,1461,117603.342225
1,1462,151876.110820
2,1463,171931.383684
3,1464,193247.205207
4,1465,178877.334967
5,1466,163561.722073
6,1467,160591.615305
7,1468,157911.945425
8,1469,189655.970602
9,1470,119880.973170
